<a href="https://colab.research.google.com/github/mehr64/Predicting-Crime-Trends/blob/main/MyProject_Hive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Crime Analysis").getOrCreate()
df = spark.read.csv("/content/sample_data/cleaned_data.csv", header=True, inferSchema=True)

In [ ]:
#Examine the overall changes in the number of crimes in different quarters (Financial Quarter and Number of Offences).
from pyspark.sql.functions import col

df.groupBy("Financial Quarter").sum("Number of Offences").orderBy("sum(Number of Offences)", ascending=False).show()

+-----------------+-----------------------+
|Financial Quarter|sum(Number of Offences)|
+-----------------+-----------------------+
|                1|                6459000|
|                2|                5397100|
|                3|                5154518|
|                4|                4983585|
+-----------------+-----------------------+



 1.How do crime rates vary across different financial quarters?

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Crime Analysis") \
    .enableHiveSupport() \
    .getOrCreate()

df = spark.read.csv("/content/sample_data/cleaned_data.csv", header=True, inferSchema=True)

# Group by "Financial Quarter" and sum "Number of Offences"
df_summed = df.groupBy("Financial Quarter").agg(sum("Number of Offences").alias("total_offences"))

# Order by the sum of "Number of Offences" in descending order
df_summed_ordered = df_summed.orderBy(col("total_offences"), ascending=False)

# Show the result
df_summed_ordered.show()

# Stop the Spark session
spark.stop()

+-----------------+--------------+
|Financial Quarter|total_offences|
+-----------------+--------------+
|                1|       6459000|
|                2|       5397100|
|                3|       5154518|
|                4|       4983585|
+-----------------+--------------+



2.Which police force areas have seen the biggest shifts in crime rates over the course of
various fiscal quarters?


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Crime Analysis") \
    .enableHiveSupport() \
    .getOrCreate()

df = spark.read.csv("/content/sample_data/cleaned_data.csv", header=True, inferSchema=True)

# Group by "Financial Quarter" and "Force Name", and sum "Number of Offences"
df_summed = df.groupBy("Financial Quarter", "Force Name").agg(sum("Number of Offences").alias("total_offences"))

# Order the results by "Financial Quarter" and the sum of "Number of Offences" in descending order
df_summed_ordered = df_summed.orderBy("Financial Quarter", col("total_offences"), ascending=False)

# Show the result
df_summed_ordered.show()

# Stop the Spark session
spark.stop()

+-----------------+-------------------+--------------+
|Financial Quarter|         Force Name|total_offences|
+-----------------+-------------------+--------------+
|                4|Metropolitan Police|        812573|
|                4| Greater Manchester|        322915|
|                4|      West Midlands|        319177|
|                4|     West Yorkshire|        280508|
|                4|               Kent|        166679|
|                4|      Thames Valley|        163953|
|                4|          Hampshire|        155365|
|                4|              Essex|        153834|
|                4|         Merseyside|        147251|
|                4|    South Yorkshire|        144707|
|                4|        Northumbria|        135237|
|                4|  Avon and Somerset|        129948|
|                4|         Lancashire|        125331|
|                4|             Sussex|        114299|
|                4|        South Wales|         99271|
|         